# Topic Similarity: with LDA TF-IDF model & Cossine similarity

## Import Packages

In [1]:
import numpy as np
import pandas as pd

import spacy
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.matutils import cossim
from gensim.models import CoherenceModel
from gensim import corpora, models

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from pprint import pprint

import os
import re

import pyLDAvis.gensim
import pickle 
import pyLDAvis


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Read Data

In [2]:
df_First = pd.read_excel('Data_files/New_1st_debate.xlsx', engine="openpyxl")
df_Second = pd.read_excel('Data_files/New_2nd_debate.xlsx', engine="openpyxl")
#df_Second = pd.read_csv('Data_files/2020_second_debate_final.csv', encoding = "utf-8")

df_All = df_First.append(df_Second)
print(df_First.shape)
print(df_Second.shape)
print(df_All.shape)

(789, 5)
(514, 3)
(1303, 5)


## Data Processing

In [3]:
# standard stopwords
my_stopwords = stopwords.words('english')
# additional stopwords
my_stopwords.extend(['from', 'subject', 're', 'edu', 'use'])

# Remove punctuation
df_First['text_processed'] = df_First['text'].map(lambda x: re.sub('[,.!?]', '', str(x)))
df_Second['text_processed'] = df_Second['text'].map(lambda x: re.sub('[,.!?]', '', str(x)))
df_All['text_processed'] = df_All['text'].map(lambda x: re.sub('[,.!?]', '', str(x)))

# Convert lowercase
df_First['text_processed'] = df_First['text_processed'].map(lambda x: x.lower())
df_Second['text_processed'] = df_Second['text_processed'].map(lambda x: x.lower())
df_All['text_processed'] = df_All['text_processed'].map(lambda x: x.lower())

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


dataF = df_First.text_processed.values.tolist()
data_words_F = list(sent_to_words(dataF))
data_words_F = [[x for x in word if len(x) > 5] for word in data_words_F]
data_words_F

[['evening',
  'health',
  'education',
  'campus',
  'western',
  'reserve',
  'university',
  'cleveland',
  'clinic',
  'wallace',
  'welcome',
  'presidential',
  'debates',
  'between',
  'president',
  'donald',
  'former',
  'president',
  'debate',
  'sponsored',
  'commission',
  'presidential',
  'debates',
  'commission',
  'designed',
  'format',
  'roughly',
  'minute',
  'segments',
  'minute',
  'answers',
  'candidate',
  'question',
  'discussion',
  'segment',
  'campaigns',
  'agreed',
  'record',
  'decided',
  'topics',
  'questions',
  'assure',
  'questions',
  'shared',
  'commission',
  'candidates'],
 ['debate',
  'conducted',
  'health',
  'safety',
  'protocols',
  'designed',
  'cleveland',
  'clinic',
  'serving',
  'health',
  'security',
  'advisor',
  'commission',
  'debates',
  'precaution',
  'campaigns',
  'agreed',
  'candidates',
  'beginning',
  'tonight',
  'debate',
  'audience',
  'promised',
  'remain',
  'silent',
  'cheers',
  'interruption

In [5]:
dataS = df_Second.text_processed.values.tolist()
data_words_S = list(sent_to_words(dataS))
data_words_S = [[x for x in word if len(x) > 5] for word in data_words_S]
data_words_S

[['evening',
  'everyone',
  'evening',
  'moderate',
  'debate',
  'tonight',
  'debate',
  'welcome',
  'family',
  'thankful',
  'feeling',
  'better',
  'welcome',
  'family',
  'tonight',
  'excited',
  'looking',
  'forward',
  'really',
  'robust',
  'discussion',
  'reiterate',
  'guidelines',
  'candidates',
  'talking',
  'please',
  'applause',
  'reactions',
  'except',
  'course',
  'applause',
  'everyone',
  'having',
  'really',
  'lifetime',
  'organized',
  'settled',
  'silence',
  'evening',
  'belmont',
  'university',
  'nashville',
  'tennessee',
  'kristen',
  'welker',
  'welcome',
  'presidential',
  'debate',
  'between',
  'president',
  'donald',
  'former',
  'president',
  'tonight',
  'debate',
  'sponsored',
  'commission',
  'presidential',
  'debates',
  'conducted',
  'health',
  'safety',
  'protocols',
  'designed',
  'commission',
  'health',
  'security',
  'advisor',
  'audience',
  'promised',
  'remain',
  'silent',
  'cheers',
  'interruption

In [6]:
dataA = df_All.text_processed.values.tolist()
data_words_All = list(sent_to_words(dataA))
data_words_All = [[x for x in word if len(x) > 5] for word in data_words_All]
data_words_All

[['evening',
  'health',
  'education',
  'campus',
  'western',
  'reserve',
  'university',
  'cleveland',
  'clinic',
  'wallace',
  'welcome',
  'presidential',
  'debates',
  'between',
  'president',
  'donald',
  'former',
  'president',
  'debate',
  'sponsored',
  'commission',
  'presidential',
  'debates',
  'commission',
  'designed',
  'format',
  'roughly',
  'minute',
  'segments',
  'minute',
  'answers',
  'candidate',
  'question',
  'discussion',
  'segment',
  'campaigns',
  'agreed',
  'record',
  'decided',
  'topics',
  'questions',
  'assure',
  'questions',
  'shared',
  'commission',
  'candidates'],
 ['debate',
  'conducted',
  'health',
  'safety',
  'protocols',
  'designed',
  'cleveland',
  'clinic',
  'serving',
  'health',
  'security',
  'advisor',
  'commission',
  'debates',
  'precaution',
  'campaigns',
  'agreed',
  'candidates',
  'beginning',
  'tonight',
  'debate',
  'audience',
  'promised',
  'remain',
  'silent',
  'cheers',
  'interruption

## Build the bigram and trigram models

In [7]:
# Build the bigram and trigram models
bigram_F = gensim.models.Phrases(data_words_F, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_F = gensim.models.Phrases(bigram_F[data_words_F], threshold=100)
bigram_S = gensim.models.Phrases(data_words_S, min_count=5, threshold=100) 
trigram_S = gensim.models.Phrases(bigram_S[data_words_S], threshold=100)
bigram_All = gensim.models.Phrases(data_words_All, min_count=5, threshold=100) 
trigram_All = gensim.models.Phrases(bigram_All[data_words_All], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_F = gensim.models.phrases.Phraser(bigram_F)
trigram_mod_F = gensim.models.phrases.Phraser(trigram_F)
bigram_mod_S = gensim.models.phrases.Phraser(bigram_S)
trigram_mod_S = gensim.models.phrases.Phraser(trigram_S)
bigram_mod_All = gensim.models.phrases.Phraser(bigram_All)
trigram_mod_All = gensim.models.phrases.Phraser(trigram_All)

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in my_stopwords] for doc in texts]

def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts, bigram_mod, trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
data_words_nostops_F = remove_stopwords(data_words_F)
data_words_nostops_S = remove_stopwords(data_words_S)
data_words_nostops_All = remove_stopwords(data_words_All)

# Form Bigrams
data_words_bigrams_F = make_bigrams(data_words_nostops_F, bigram_mod_F)
data_words_bigrams_S = make_bigrams(data_words_nostops_S, bigram_mod_S)
data_words_bigrams_All = make_bigrams(data_words_nostops_All, bigram_mod_All)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

#Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized_F = lemmatization(data_words_bigrams_F, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized_S = lemmatization(data_words_bigrams_S, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized_All = lemmatization(data_words_bigrams_All, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


## Create Dictionary & Corpus (bag of words & TF-IDF) for LDA model

In [10]:
# Create Dictionary
id2word_F = corpora.Dictionary(data_lemmatized_F)
id2word_S = corpora.Dictionary(data_lemmatized_S)
id2word_All = corpora.Dictionary(data_lemmatized_All)

# Create Corpus
texts_F = data_lemmatized_F
texts_S = data_lemmatized_S
texts_All = data_lemmatized_All

# Term Document Frequency
corpus_F = [id2word_F.doc2bow(text) for text in texts_F]
corpus_S = [id2word_S.doc2bow(text) for text in texts_S]
corpus_All = [id2word_All.doc2bow(text) for text in texts_All]

#print(corpus_F)

In [11]:
tfidf_F = models.TfidfModel(corpus_F)
corpus_F_tfidf = tfidf_F[corpus_F]

tfidf_S = models.TfidfModel(corpus_S)
corpus_S_tfidf = tfidf_S[corpus_S]

tfidf_All = models.TfidfModel(corpus_All)
corpus_All_tfidf = tfidf_All[corpus_All]

## Create LDA model

#### LDA topic model: based on Debate1

In [12]:
# Build First debate LDA model
lda_model_First = gensim.models.LdaMulticore(corpus=corpus_F,
                                       id2word=id2word_F,
                                       num_topics=8, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

#### LDA-tfidf topic model: based on Debate1

In [13]:
# Build First debate LDA-tfidf model
lda_model_First_tfidf = gensim.models.LdaMulticore(corpus=corpus_F_tfidf,
                                       id2word=id2word_F,
                                       num_topics=8, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

#### LDA topic model: based on Debate2

In [14]:
# Build Second debate LDA model
lda_model_Second = gensim.models.LdaMulticore(corpus=corpus_S,
                                       id2word=id2word_S,
                                       num_topics=8, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

#### LDA-tfidf topic model: based on Debate2

In [15]:
# Build Second debate LDA-tfidf model
lda_model_Second_tfidf = gensim.models.LdaMulticore(corpus=corpus_S_tfidf,
                                       id2word=id2word_S,
                                       num_topics=8, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

#### LDA topic model: based on Debate2

In [16]:
# Build Combined debate LDA model
lda_model_All = gensim.models.LdaMulticore(corpus=corpus_All,
                                       id2word=id2word_All,
                                       num_topics=8, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

#### LDA-tfidf topic model: based on Both Debates

In [17]:
# Build Combined debate LDA-tfidf model
lda_model_All_tfidf = gensim.models.LdaMulticore(corpus=corpus_All_tfidf,
                                       id2word=id2word_All,
                                       num_topics=8, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

#### Show the top topics from LDA-tfidf model trained from both debate

In [18]:
pprint(lda_model_First.print_topics())

[(0,
  '0.107*"people" + 0.036*"number" + 0.021*"ballot" + 0.017*"police" + '
  '0.017*"question" + 0.015*"debate" + 0.014*"totally" + 0.012*"obamacare" + '
  '0.010*"state" + 0.010*"honest"'),
 (1,
  '0.078*"crosstalk" + 0.043*"economy" + 0.019*"segment" + 0.019*"include" + '
  '0.016*"charge" + 0.015*"tonight" + 0.015*"president" + 0.012*"second" + '
  '0.012*"create" + 0.012*"make"'),
 (2,
  '0.035*"people" + 0.031*"question" + 0.029*"forest" + 0.019*"get" + '
  '0.018*"management" + 0.018*"administration" + 0.017*"thing" + '
  '0.016*"energy" + 0.015*"already" + 0.015*"burn"'),
 (3,
  '0.044*"people" + 0.029*"country" + 0.024*"gentleman" + 0.023*"happen" + '
  '0.022*"answer" + 0.020*"president" + 0.017*"question" + 0.017*"minute" + '
  '0.017*"record" + 0.016*"agree"'),
 (4,
  '0.050*"dollar" + 0.037*"million" + 0.033*"minute" + 0.032*"people" + '
  '0.025*"believe" + 0.021*"climate" + 0.018*"record" + 0.017*"say" + '
  '0.014*"question" + 0.012*"billion"'),
 (5,
  '0.072*"electio

In [19]:
pprint(lda_model_First_tfidf.print_topics())

[(0,
  '0.111*"crosstalk" + 0.052*"people" + 0.047*"minute" + 0.029*"talk" + '
  '0.028*"ballot" + 0.021*"number" + 0.021*"question" + 0.016*"honest" + '
  '0.015*"evidence" + 0.011*"democratic"'),
 (1,
  '0.046*"economy" + 0.016*"listen" + 0.015*"little" + 0.013*"create" + '
  '0.011*"trouble" + 0.011*"bring" + 0.011*"repeatedly" + 0.010*"charge" + '
  '0.009*"question" + 0.009*"resent"'),
 (2,
  '0.028*"radical" + 0.025*"million" + 0.021*"country" + 0.017*"election" + '
  '0.016*"administration" + 0.012*"people" + 0.011*"count" + 0.011*"person" + '
  '0.011*"deserve" + 0.010*"rig"'),
 (3,
  '0.038*"gentleman" + 0.023*"answer" + 0.016*"debate" + 0.015*"forward" + '
  '0.013*"totally" + 0.012*"urge" + 0.011*"second" + 0.011*"president" + '
  '0.010*"thing" + 0.010*"segment"'),
 (4,
  '0.068*"dollar" + 0.025*"say" + 0.018*"happen" + 0.014*"believe" + '
  '0.011*"return" + 0.011*"careful" + 0.010*"climate" + 0.009*"street" + '
  '0.009*"common" + 0.008*"science"'),
 (5,
  '0.025*"change"

In [20]:
pprint(lda_model_Second.print_topics())

[(0,
  '0.028*"fracke" + 0.018*"industry" + 0.015*"family" + 0.014*"release" + '
  '0.013*"economy" + 0.013*"immigration" + 0.012*"people" + 0.012*"happen" + '
  '0.011*"reform" + 0.011*"country"'),
 (1,
  '0.061*"question" + 0.048*"people" + 0.026*"number" + 0.025*"crosstalk" + '
  '0.024*"family" + 0.016*"president" + 0.015*"plant" + 0.013*"country" + '
  '0.011*"get" + 0.011*"want"'),
 (2,
  '0.024*"business" + 0.021*"excuse" + 0.020*"dollar" + 0.019*"second" + '
  '0.019*"happen" + 0.017*"number" + 0.017*"follow" + 0.017*"people" + '
  '0.017*"country" + 0.016*"clean"'),
 (3,
  '0.026*"talk" + 0.024*"come" + 0.019*"call" + 0.018*"child" + '
  '0.017*"president" + 0.016*"energy" + 0.013*"people" + 0.013*"happen" + '
  '0.012*"understand" + 0.012*"parent"'),
 (4,
  '0.041*"statement" + 0.035*"quickly" + 0.031*"section" + 0.020*"business" + '
  '0.019*"big" + 0.014*"number" + 0.012*"people" + 0.011*"happen" + '
  '0.011*"state" + 0.010*"already"'),
 (5,
  '0.037*"people" + 0.021*"insu

In [21]:
pprint(lda_model_Second_tfidf.print_topics())

[(0,
  '0.051*"crosstalk" + 0.029*"industry" + 0.016*"thing" + 0.013*"happen" + '
  '0.010*"destroy" + 0.009*"family" + 0.008*"transition" + 0.007*"ban" + '
  '0.007*"understand" + 0.007*"people"'),
 (1,
  '0.014*"family" + 0.012*"totally" + 0.009*"company" + 0.008*"recently" + '
  '0.007*"chance" + 0.007*"character" + 0.007*"separate" + 0.007*"business" + '
  '0.006*"grow" + 0.006*"energy"'),
 (2,
  '0.033*"excuse" + 0.029*"follow" + 0.018*"fracke" + 0.016*"remember" + '
  '0.012*"respond" + 0.012*"website" + 0.007*"country" + 0.007*"disaster" + '
  '0.006*"believe" + 0.006*"people"'),
 (3,
  '0.094*"question" + 0.018*"call" + 0.008*"big" + 0.007*"prison" + '
  '0.007*"president" + 0.007*"exactly" + 0.007*"communitynow" + '
  '0.007*"relationship" + 0.006*"police" + 0.006*"horrible"'),
 (4,
  '0.038*"quickly" + 0.031*"statement" + 0.027*"section" + 0.022*"number" + '
  '0.009*"already" + 0.008*"topic" + 0.008*"immigration" + 0.008*"reference" + '
  '0.008*"finish" + 0.007*"people"'),


In [22]:
pprint(lda_model_All.print_topics())

[(0,
  '0.075*"question" + 0.057*"family" + 0.042*"people" + 0.032*"talk" + '
  '0.016*"month" + 0.015*"parent" + 0.015*"thousand" + 0.013*"accord" + '
  '0.012*"immigration" + 0.012*"section"'),
 (1,
  '0.046*"dollar" + 0.043*"number" + 0.025*"million" + 0.024*"create" + '
  '0.020*"get" + 0.018*"market" + 0.018*"minute" + 0.016*"pay" + '
  '0.016*"president" + 0.015*"building"'),
 (2,
  '0.059*"crosstalk" + 0.034*"people" + 0.025*"country" + 0.024*"second" + '
  '0.023*"excuse" + 0.022*"option" + 0.021*"talk" + 0.017*"problem" + '
  '0.017*"insurance" + 0.017*"destroy"'),
 (3,
  '0.077*"people" + 0.052*"happen" + 0.037*"business" + 0.020*"industry" + '
  '0.018*"country" + 0.015*"school" + 0.014*"person" + 0.014*"response" + '
  '0.013*"place" + 0.012*"think"'),
 (4,
  '0.052*"fracke" + 0.029*"statement" + 0.024*"thing" + 0.024*"want" + '
  '0.017*"vaccine" + 0.017*"finish" + 0.014*"answer" + 0.014*"happen" + '
  '0.014*"transition" + 0.013*"social"'),
 (5,
  '0.032*"people" + 0.028*

In [23]:
# Print the Keyword in the topics
pprint(lda_model_All_tfidf.print_topics())

[(0,
  '0.028*"family" + 0.025*"ballot" + 0.013*"change" + 0.012*"immigration" + '
  '0.011*"course" + 0.011*"honest" + 0.010*"website" + 0.010*"former" + '
  '0.009*"topic" + 0.008*"military"'),
 (1,
  '0.037*"quickly" + 0.031*"minute" + 0.028*"follow" + 0.019*"finish" + '
  '0.018*"totally" + 0.013*"respond" + 0.013*"radical" + 0.011*"release" + '
  '0.010*"state" + 0.010*"president"'),
 (2,
  '0.062*"crosstalk" + 0.026*"excuse" + 0.024*"second" + 0.019*"talk" + '
  '0.018*"statement" + 0.018*"country" + 0.014*"thing" + 0.012*"really" + '
  '0.012*"people" + 0.012*"million"'),
 (3,
  '0.043*"people" + 0.021*"section" + 0.019*"happen" + 0.016*"call" + '
  '0.016*"simply" + 0.016*"business" + 0.016*"answer" + 0.013*"question" + '
  '0.011*"administration" + 0.010*"understand"'),
 (4,
  '0.015*"month" + 0.014*"single" + 0.013*"big" + 0.013*"kid" + 0.013*"want" + '
  '0.012*"scientist" + 0.010*"think" + 0.010*"vaccine" + 0.009*"number" + '
  '0.009*"place"'),
 (5,
  '0.080*"question" + 0

## Visualize topics-keywords of LDA

### Topic: First debate (BoW)

In [24]:
# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_First, corpus_F, id2word_F)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.048764 -0.035279       1        1  15.687593
3     -0.018543  0.034164       2        1  15.565989
6     -0.032733 -0.020152       3        1  13.847534
5      0.200056 -0.149404       4        1  13.620135
2      0.017320  0.041161       5        1  12.247346
4      0.072983  0.226930       6        1  10.978482
1     -0.136845 -0.028400       7        1   9.926355
7     -0.151002 -0.069020       8        1   8.126562, topic_info=           Term       Freq      Total Category  logprob  loglift
51     election  29.000000  29.000000  Default  30.0000  30.0000
215   crosstalk  37.000000  37.000000  Default  29.0000  29.0000
153      ballot  36.000000  36.000000  Default  28.0000  28.0000
74       happen  34.000000  34.000000  Default  27.0000  27.0000
464      dollar  16.000000  16.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
183       thing   2.293017  13.960500   Topic8  -4.6250   0.7037
350      market   1.670995   4.557338   Topic8  -4.9414   1.5067
461     federal   1.745640   5.969049   Topic8  -4.8977   1.2806
232  understand   1.500211   6.027070   Topic8  -5.0492   1.1194
85      already   1.564175  10.490153   Topic8  -5.0075   0.6070

[425 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
167       3  0.199187  absolutely
167       5  0.597560  absolutely
800       3  0.494376      absorb
688       4  0.665100      accept
142       4  0.910505      accord
...     ...       ...         ...
298       5  0.330254        work
298       8  0.660508        work
902       7  0.688694      worker
400       1  0.741324     worried
114       2  0.678628       write

[566 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 7, 6, 3, 5, 2, 8])

### Topic: First debate (TF-IDF)

In [25]:
# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_First_tfidf, corpus_F_tfidf, id2word_F)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.054943 -0.147063       1        1  16.038774
3     -0.048028  0.010892       2        1  15.569994
0      0.168110  0.071073       3        1  13.972217
2     -0.051047  0.017452       4        1  11.852976
5     -0.012376  0.002246       5        1  11.038154
4     -0.052306  0.048611       6        1  10.895176
1     -0.028214  0.005806       7        1  10.732395
7     -0.031082 -0.009017       8        1   9.900314, topic_info=          Term       Freq      Total Category  logprob  loglift
215  crosstalk  15.000000  15.000000  Default  30.0000  30.0000
464     dollar   7.000000   7.000000  Default  29.0000  29.0000
13      minute   6.000000   6.000000  Default  28.0000  28.0000
246    support   6.000000   6.000000  Default  27.0000  27.0000
301    economy   5.000000   5.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
248     become   0.730542   2.199605   Topic8  -4.8549   1.2104
133   increase   0.431278   1.096291   Topic8  -5.3820   1.3797
222  president   0.804029   3.823835   Topic8  -4.7591   0.7532
183      thing   0.529046   2.684269   Topic8  -5.1776   0.6885
152       talk   0.648000   5.830896   Topic8  -4.9748   0.1156

[346 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
167       1  0.554531      absolutely
688       4  0.884583          accept
142       5  0.715938          accord
451       4  0.898017  administration
233       1  0.506812          almost
...     ...       ...             ...
247       1  0.826362         willing
901       1  0.909737          winner
298       5  0.860043            work
400       3  1.011987         worried
114       2  0.891216           write

[250 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 1, 3, 6, 5, 2, 8])

### Topic: Second debate (BoW)

In [26]:
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_Second, corpus_S, id2word_S)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.148777  0.116191       1        1  16.482294
5     -0.076965 -0.092340       2        1  15.731886
7      0.134508 -0.024988       3        1  13.391040
0      0.028441  0.021514       4        1  12.848242
6     -0.033830 -0.135451       5        1  12.820354
2      0.093917  0.104249       6        1  12.583317
3     -0.025209  0.032874       7        1   9.702067
4      0.027916 -0.022049       8        1   6.440805, topic_info=          Term       Freq      Total Category  logprob  loglift
216   question  33.000000  33.000000  Default  30.0000  30.0000
721     change  17.000000  17.000000  Default  29.0000  29.0000
445  statement   9.000000   9.000000  Default  28.0000  28.0000
626     fracke  15.000000  15.000000  Default  27.0000  27.0000
139    quickly   9.000000   9.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
58      number   2.621048  29.673620   Topic8  -4.2646   0.3158
196     happen   2.071921  23.737612   Topic8  -4.4997   0.3039
75      people   2.152037  70.023613   Topic8  -4.4618  -0.7399
52     country   1.527593  30.677708   Topic8  -4.8045  -0.2573
85     vaccine   1.046272   4.840649   Topic8  -5.1830   1.2107

[440 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
187       4  0.729614     access
187       6  0.364807     access
647       6  0.763330     accord
421       1  0.866729    account
863       7  0.824758     action
...     ...       ...        ...
323       8  0.800426  wonderful
142       6  0.657067       work
142       8  0.328533       work
625       2  0.949786      worry
324       8  0.800426       wrap

[668 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 6, 8, 1, 7, 3, 4, 5])

### Topic: Second debate (TF-IDF)

In [27]:
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_Second_tfidf, corpus_S_tfidf, id2word_S)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.058950 -0.083978       1        1  14.281384
6     -0.035470  0.033223       2        1  14.017552
7      0.046547  0.008675       3        1  12.960047
0     -0.043374  0.068287       4        1  12.948055
3      0.085194 -0.010012       5        1  12.933941
5      0.017963 -0.019606       6        1  11.296156
1     -0.004532  0.005754       7        1  10.982185
4     -0.007377 -0.002343       8        1  10.580677, topic_info=          Term       Freq      Total Category  logprob  loglift
216   question  11.000000  11.000000  Default  30.0000  30.0000
358  crosstalk   5.000000   5.000000  Default  29.0000  29.0000
217     second   4.000000   4.000000  Default  28.0000  28.0000
139    quickly   3.000000   3.000000  Default  27.0000  27.0000
222     excuse   4.000000   4.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
75      people   0.590946   4.733921   Topic8  -4.9471   0.1654
115   business   0.349971   2.325402   Topic8  -5.4709   0.3523
636    destroy   0.323093   2.135162   Topic8  -5.5508   0.3578
12      family   0.329704   3.745534   Topic8  -5.5306  -0.1840
52     country   0.309268   3.131544   Topic8  -5.5946  -0.0689

[389 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
874       3  0.835521     address
960       1  0.942757       agree
189       8  0.752593     already
755       5  1.016989     anymore
168       4  0.902993         ban
...     ...       ...         ...
166       7  0.656238     totally
934       4  0.827734  transition
449       1  0.819072       treat
178       4  0.623345  understand
807       1  0.541710     website

[125 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 7, 8, 1, 4, 6, 2, 5])

### Topic: First & Second debate (BoW)

In [28]:
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_All, corpus_All, id2word_All)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.091860 -0.087060       1        1  17.689266
0     -0.117368 -0.015283       2        1  14.032711
2     -0.092650  0.085032       3        1  12.832536
1      0.213778  0.143473       4        1  12.452069
5      0.005009  0.112590       5        1  12.435621
6      0.135135 -0.210415       6        1  11.591177
7     -0.097614  0.023902       7        1  10.618024
4      0.045570 -0.052241       8        1   8.348590, topic_info=          Term       Freq       Total Category  logprob  loglift
15    question  78.000000   78.000000  Default  30.0000  30.0000
188     family  46.000000   46.000000  Default  29.0000  29.0000
213  crosstalk  45.000000   45.000000  Default  28.0000  28.0000
74      happen  66.000000   66.000000  Default  27.0000  27.0000
460     dollar  33.000000   33.000000  Default  26.0000  26.0000
..         ...        ...         ...      ...      ...      ...
154       talk   5.471132   50.246902   Topic8  -4.4695   0.2656
49         big   3.450539   11.968369   Topic8  -4.9305   1.2393
45      people   4.985985  185.758682   Topic8  -4.5624  -1.1347
51    election   3.644265   32.808002   Topic8  -4.8758   0.2856
127     number   3.819513   60.111370   Topic8  -4.8289  -0.2730

[407 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
361       1  0.866114     ability
167       4  0.757400  absolutely
167       6  0.189350  absolutely
144       2  0.904768      accord
1383      2  0.667962      action
...     ...       ...         ...
217       6  0.053505        well
217       7  0.695566        well
244       5  0.901663     willing
1413      4  0.918273      window
1276      3  0.653353     working

[555 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2, 6, 7, 8, 5])

### Topic: First & Second debate (TF-IDF)

In [29]:
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_All_tfidf, corpus_All_tfidf, id2word_All)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.199981  0.014116       1        1  20.384886
7     -0.037261 -0.085243       2        1  14.171799
5     -0.054739  0.153015       3        1  12.878922
3     -0.008472  0.024325       4        1  12.169658
1     -0.026231 -0.006819       5        1  10.823473
0     -0.016504 -0.049217       6        1  10.498535
6     -0.035643 -0.036606       7        1   9.747004
4     -0.021132 -0.013571       8        1   9.325727, topic_info=          Term       Freq      Total Category  logprob  loglift
213  crosstalk  22.000000  22.000000  Default  30.0000  30.0000
15    question  22.000000  22.000000  Default  29.0000  29.0000
460     dollar   8.000000   8.000000  Default  28.0000  28.0000
43   gentleman   7.000000   7.000000  Default  27.0000  27.0000
357    quickly   7.000000   7.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
139        say   1.015809   6.894865   Topic8  -5.0631   0.4573
54       great   0.697601   1.799618   Topic8  -5.4389   1.4247
442       make   0.696935   2.509904   Topic8  -5.4399   1.0911
45      people   0.782201  18.621582   Topic8  -5.3244  -0.7976
184      thing   0.728719   6.041325   Topic8  -5.3953   0.2573

[343 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
167       5  0.891304      absolutely
144       6  0.612637          accord
610       4  0.567304         address
447       4  0.698596  administration
96        2  0.662305      affordable
...     ...       ...             ...
1342      6  0.850613         website
217       2  0.784220            well
244       3  0.727329         willing
294       4  0.651734            work
116       2  0.736613           write

[347 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 8, 6, 4, 2, 1, 7, 5])

## Evaluate LDA model

In [30]:
# Compute Coherence Score
coherence_model_lda_First = CoherenceModel(model=lda_model_First, texts=data_lemmatized_F, dictionary=id2word_F, coherence='c_v')
coherence_lda_F = coherence_model_lda_First.get_coherence()
print('Coherence Score for First debates         : ', coherence_lda_F)
coherence_model_lda_Second = CoherenceModel(model=lda_model_Second, texts=data_lemmatized_S, dictionary=id2word_S, coherence='c_v')
coherence_lda_S = coherence_model_lda_Second.get_coherence()
print('Coherence Score for Second debates        : ', coherence_lda_S)
coherence_model_lda_All = CoherenceModel(model=lda_model_All, texts=data_lemmatized_All, dictionary=id2word_All, coherence='c_v')
coherence_lda_All = coherence_model_lda_All.get_coherence()
print('Coherence Score for All debates           : ', coherence_lda_All)
print('               ----------------------------')

coherence_model_lda_First_tfidf = CoherenceModel(model=lda_model_First_tfidf, texts=data_lemmatized_F, dictionary=id2word_F, coherence='c_v')
coherence_lda_F_tfidf = coherence_model_lda_First_tfidf.get_coherence()
print('Coherence Score for First debates (TF-IDF): ', coherence_lda_F_tfidf)

coherence_model_lda_Second_tfidf = CoherenceModel(model=lda_model_Second_tfidf, texts=data_lemmatized_S, dictionary=id2word_S, coherence='c_v')
coherence_lda_S_tfidf = coherence_model_lda_Second_tfidf.get_coherence()
print('Coherence Score for Second debates(TF-IDF): ', coherence_lda_S_tfidf)

coherence_model_lda_All_tfidf = CoherenceModel(model=lda_model_All_tfidf, texts=data_lemmatized_All, dictionary=id2word_All, coherence='c_v')
coherence_lda_All_tfidf = coherence_model_lda_All_tfidf.get_coherence()
print('Coherence Score for All debates   (TF-IDF): ', coherence_lda_All_tfidf)

Coherence Score for First debates         :  0.44895445813662693
Coherence Score for Second debates        :  0.44414931929356355
Coherence Score for All debates           :  0.43474762450259635
               ----------------------------
Coherence Score for First debates (TF-IDF):  0.5940719542393509
Coherence Score for Second debates(TF-IDF):  0.6217237171297948
Coherence Score for All debates   (TF-IDF):  0.5578770421921299


## Creat documents for caculate the topic similarity

In [31]:
df_T16=pd.read_csv('Data_files/Trump_2016.csv')

# Remove punctuation
df_T16['text_processed'] = df_T16['text'].map(lambda x: re.sub('[,.!?]', '', x))
# Convert lowercase
df_T16['text_processed'] = df_T16['text_processed'].map(lambda x: x.lower())


df_T20_1 = df_First[df_First.speaker=='Donald Trump']
df_T20_1.index = np.arange(0,len(df_T20_1))

df_T20_2 = df_Second[df_Second.speaker=='Donald Trump']
df_T20_2.index = np.arange(0,len(df_T20_2))

df_B20_1 = df_First[df_First.speaker=='Joe Biden']
df_B20_1.index = np.arange(0,len(df_B20_1))

df_B20_2 = df_Second[df_Second.speaker=='Joe Biden']
df_B20_2.index = np.arange(0,len(df_B20_2))

df_T16_1 = df_T16.iloc[:177]
df_T16_2 = df_T16.iloc[177:]

text_DT_2020_1 = " ".join(txt for txt in df_T20_1.text_processed)
text_DT_2020_2 = " ".join(txt for txt in df_T20_2.text_processed)
text_JB_2020_1 = " ".join(txt for txt in df_B20_1.text_processed)
text_JB_2020_2 = " ".join(txt for txt in df_B20_2.text_processed)
text_DT_2016_1 = " ".join(txt for txt in df_T16_1.text_processed)
text_DT_2016_2 = " ".join(txt for txt in df_T16_2.text_processed)

## Topic similarity

#### Compute similarity of all Trump 2020 & Biden 2020 speach to 2020 Debate 1's topic

In [32]:
# Model: Debate 1, T vs B
test_doc = [text_DT_2020_1,text_DT_2020_2]
test_doc = [doc.split() for doc in test_doc]
test_corpus = [id2word_F.doc2bow(doc) for doc in test_doc]

# Cossim
doc1 = lda_model_First.get_document_topics(test_corpus[0], minimum_probability=0.1)
doc2 = lda_model_First.get_document_topics(test_corpus[1], minimum_probability=0.1)
print("Model: Debate 1, Trump 2020")
print(cossim(doc1, doc2))

test_doc2 = [text_JB_2020_1,text_JB_2020_2]
test_doc2 = [doc.split() for doc in test_doc2]
test_corpus2 = [id2word_F.doc2bow(doc) for doc in test_doc2]

doc3 = lda_model_First.get_document_topics(test_corpus2[0], minimum_probability=0.1)
doc4 = lda_model_First.get_document_topics(test_corpus2[1], minimum_probability=0.1)
print("Model: Debate 1, Biden 2020")
print(cossim(doc3, doc4))

Model: Debate 1, Trump 2020
0.9413847730839505
Model: Debate 1, Biden 2020
0.9340717073832174


In [33]:
# Model: Debate 1, T vs B
test_doc = [text_DT_2020_1,text_DT_2020_2]
test_doc = [doc.split() for doc in test_doc]
test_corpus = [id2word_F.doc2bow(doc) for doc in test_doc]
tfidf = models.TfidfModel(test_corpus)
corpus_tfidf = tfidf[test_corpus]

# Cossim
doc1 = lda_model_First_tfidf.get_document_topics(corpus_tfidf[0], minimum_probability=0.1)
doc2 = lda_model_First_tfidf.get_document_topics(corpus_tfidf[1], minimum_probability=0.1)
print("Model: Debate 1, Trump 2020")
print(cossim(doc1, doc2))

test_doc2 = [text_JB_2020_1,text_JB_2020_2]
test_doc2 = [doc.split() for doc in test_doc2]
test_corpus2 = [id2word_F.doc2bow(doc) for doc in test_doc2]
tfidf2 = models.TfidfModel(test_corpus2)
corpus_tfidf2 = tfidf[test_corpus2]

doc3 = lda_model_First_tfidf.get_document_topics(corpus_tfidf2[0], minimum_probability=0.1)
doc4 = lda_model_First_tfidf.get_document_topics(corpus_tfidf2[1], minimum_probability=0.1)
print("Model: Debate 1, Biden 2020")
print(cossim(doc3, doc4))

Model: Debate 1, Trump 2020
0.5472543938097627
Model: Debate 1, Biden 2020
0.8323101062481849


#### Compute similarity of all Trump 2020 & Biden 2020 speach to 2020 Debate 2's topic

In [34]:
# Model: Debate 2, T vs B
test_doc = [text_DT_2020_1,text_DT_2020_2]
test_doc = [doc.split() for doc in test_doc]
test_corpus = [id2word_F.doc2bow(doc) for doc in test_doc]
tfidf = models.TfidfModel(test_corpus)
corpus_tfidf = tfidf[test_corpus]

# Cossim
doc1 = lda_model_Second_tfidf.get_document_topics(corpus_tfidf[0], minimum_probability=0.1)
doc2 = lda_model_Second_tfidf.get_document_topics(corpus_tfidf[1], minimum_probability=0.1)
print("Model: Debate 2, Trump 2020")
print(cossim(doc1, doc2))

test_doc2 = [text_JB_2020_1,text_JB_2020_2]
test_doc2 = [doc.split() for doc in test_doc2]
test_corpus2 = [id2word_F.doc2bow(doc) for doc in test_doc2]
tfidf2 = models.TfidfModel(test_corpus2)
corpus_tfidf2 = tfidf[test_corpus2]

doc3 = lda_model_Second_tfidf.get_document_topics(corpus_tfidf2[0], minimum_probability=0.1)
doc4 = lda_model_Second_tfidf.get_document_topics(corpus_tfidf2[1], minimum_probability=0.1)
print("Model: Debate 2, Biden 2020")
print(cossim(doc3, doc4))

Model: Debate 2, Trump 2020
0.5649653500295005
Model: Debate 2, Biden 2020
0.8127361389152377


#### Compute similarity of all Trump 2020 & Biden 2020 speach to 2020 All Debate's topic

In [35]:
# Model: Debate all, T vs B
test_doc = [text_DT_2020_1,text_DT_2020_2]
test_doc = [doc.split() for doc in test_doc]
test_corpus = [id2word_All.doc2bow(doc) for doc in test_doc]
tfidf = models.TfidfModel(test_corpus)
corpus_tfidf = tfidf[test_corpus]

# Cossim
doc1 = lda_model_All_tfidf.get_document_topics(corpus_tfidf[0], minimum_probability=0.1)
doc2 = lda_model_All_tfidf.get_document_topics(corpus_tfidf[1], minimum_probability=0.1)
print("Model: Debate all, Trump 2020")
print(cossim(doc1, doc2))

test_doc2 = [text_JB_2020_1,text_JB_2020_2]
test_doc2 = [doc.split() for doc in test_doc2]
test_corpus2 = [id2word_All.doc2bow(doc) for doc in test_doc2]
tfidf2 = models.TfidfModel(test_corpus2)
corpus_tfidf2 = tfidf[test_corpus2]

doc3 = lda_model_All_tfidf.get_document_topics(corpus_tfidf2[0], minimum_probability=0.1)
doc4 = lda_model_All_tfidf.get_document_topics(corpus_tfidf2[1], minimum_probability=0.1)
print("Model: Debate all, Biden 2020")
print(cossim(doc3, doc4))

Model: Debate all, Trump 2020
0.6372831731449873
Model: Debate all, Biden 2020
0.8409249251379914


#### Compute similarity of all Trump 2020 & Trump 2016 speach to 2020 All Debate's topic

In [36]:
# Model: Debate all, T16 vs T20
test_doc = [text_DT_2020_1,text_DT_2020_2]
test_doc = [doc.split() for doc in test_doc]
test_corpus = [id2word_All.doc2bow(doc) for doc in test_doc]
tfidf = models.TfidfModel(test_corpus)
corpus_tfidf = tfidf[test_corpus]

# Cossim
doc1 = lda_model_All_tfidf.get_document_topics(corpus_tfidf[0], minimum_probability=0)
doc2 = lda_model_All_tfidf.get_document_topics(corpus_tfidf[1], minimum_probability=0)
print("Model: Debate all, Trump 2020")
print(cossim(doc1, doc2))

test_doc2 = [text_DT_2016_1,text_DT_2016_2]
test_doc2 = [doc.split() for doc in test_doc2]
test_corpus2 = [id2word_All.doc2bow(doc) for doc in test_doc2]
tfidf2 = models.TfidfModel(test_corpus2)
corpus_tfidf2 = tfidf[test_corpus2]

doc3 = lda_model_All_tfidf.get_document_topics(corpus_tfidf2[0], minimum_probability=0)
doc4 = lda_model_All_tfidf.get_document_topics(corpus_tfidf2[1], minimum_probability=0)
print("Model: Debate all, Trump 2016")
print(cossim(doc3, doc4))

Model: Debate all, Trump 2020
0.8277595736942694
Model: Debate all, Trump 2016
0.9719217700029001
